# I523: Fall17 - Final Project
###   By HID333;HID337

## IoT and Big Data Analytics for Equipment Predictive Health Management (PHM)
### Project Description:

## Overview
* Read Equipment Sensor Training Data from ORACLE DB tables
* Build KNN Classifier 
  * Find best K value for KNN
  * Build KNN model using Equipment Sensor Training Data
  * Find the accurancy of the Model
* Read Sensor Data which needs to be classified using KNN
* Run Predictions on the Test Sensor Data
* Update the results back in the Oracle DB
* Run Analytics on the updated results
* Repeat above steps for Equipment2 and Equipment3

### Step 1: Read Training Data from Amazon RDS (Oracle DB)

In [247]:
#import regular python and oracle libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


#import oracle library
import cx_Oracle

#Get amazon RDS oracle db connection to read Training and Test Data
connection = cx_Oracle.connect('tdmaster/telidos2015@tdmetadata.ckevkzrwi1v2.us-west-2.rds.amazonaws.com:1521/orcl')
cursor = connection.cursor()

#define sql statements to select data for each equipment
eqp1SQL="SELECT INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS,LABEL FROM SENSOR_TRAIN_DATA WHERE UPPER(EQP_NAME)=UPPER('Vacuum Blower')"
eqp2SQL="SELECT INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS,LABEL FROM SENSOR_TRAIN_DATA WHERE UPPER(EQP_NAME)=UPPER('Smart Valve')"
eqp3SQL="SELECT INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS,LABEL FROM SENSOR_TRAIN_DATA WHERE UPPER(EQP_NAME)=UPPER('Air Compressor')"


### Step 2: Build KNN Classifier

In [248]:
#Though Equipments are using same features, but noramal operation parameters are varying for each equipment.
#One Model trained for one equipment cannot be used to predicit health of another equipment. 
#Since we are supporting 3 equipments, building 3 KNN models one for each equipment.

#Build EQP1 Model: Vacuum Blower
cursor.execute(eqp1SQL)
eqp1SQLResult = cursor.fetchall() 
eqp1List = [list(i) for i in eqp1SQLResult]
eqp1TrainArray = np.asarray(eqp1List)
# Split-out validation dataset for EQP1
x1 = eqp1TrainArray[:,0:7]
y1 = eqp1TrainArray[:,7]
#random_state can be any number. This is to replicate results over multiple runs
x1_train, x1_validation, y1_train, y1_validation = model_selection.train_test_split(x1, y1, test_size=.2, random_state=7)


#### View first fiew rows

In [249]:
#describe data first few rows
df = pd.DataFrame(eqp1SQLResult,columns = ['INT_AMB_TEMP','EXT_AMB_TEMP','INPUT_CUR','INPUT_VOLT','VIB_X_AXIS','VIB_Y_AXIS','VIB_Z_AXIS','LABEL'])
print(df.head())
#end describe data

   INT_AMB_TEMP  EXT_AMB_TEMP  INPUT_CUR  INPUT_VOLT  VIB_X_AXIS  VIB_Y_AXIS  \
0          40.0          30.0       12.0      129.39    2.696955   -0.894902   
1          40.0          30.0       12.0      131.39    2.696955   -0.894902   
2          40.0          30.0       12.0      133.39    2.696955   -0.894902   
3          40.0          30.0       12.0      135.39    2.696955   -0.894902   
4          40.0          30.0       12.0      137.39    2.696955   -0.894902   

   VIB_Z_AXIS                     LABEL  
0   -3.533973  INPUT_OVER_VOLT_FAULT_OP  
1   -3.533973  INPUT_OVER_VOLT_FAULT_OP  
2   -3.533973  INPUT_OVER_VOLT_FAULT_OP  
3   -3.533973  INPUT_OVER_VOLT_FAULT_OP  
4   -3.533973  INPUT_OVER_VOLT_FAULT_OP  


#### View last few rows

In [250]:
print(df.tail())

      INT_AMB_TEMP  EXT_AMB_TEMP  INPUT_CUR  INPUT_VOLT  VIB_X_AXIS  \
1123          40.0          45.0       12.0     119.615    2.696955   
1124          40.0          45.0       12.0     121.615    2.696955   
1125          90.0          50.0       18.0     120.000    6.717567   
1126          90.0          50.0       18.0     120.000    7.455679   
1127          90.0          50.0       18.0     120.000    7.455679   

      VIB_Y_AXIS  VIB_Z_AXIS                 LABEL  
1123   -0.894902   -3.533973  ABNORMAL_OP_30_DEG_C  
1124   -0.894902   -3.533973  ABNORMAL_OP_30_DEG_C  
1125   -3.678868   -9.678345    BEARING_DEGRADE_OP  
1126   -3.314660   -9.678345    BEARING_DEGRADE_OP  
1127   -3.678868   -8.720189    BEARING_DEGRADE_OP  


#### Find best value of K

In [251]:
# find best value of K for KNN model
k_range = list(range(1, 10))
# list different accuracy scores.
k_scores = []

for k in k_range:
    # use 10-fold cross validation
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn,x1, y1, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())

# get best k
best_k = k_range[k_scores.index(max(k_scores))]
print ("The best value for K is %d" % best_k)

# build KNN with best K
eqp1KNN = KNeighborsClassifier(n_neighbors=best_k)
eqp1KNN.fit(x1_train, y1_train)

The best value for K is 8


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=8, p=2,
           weights='uniform')

#### Find Accuracy of the Model

In [252]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
cv_results = model_selection.cross_val_score(eqp1KNN, x1_train, y1_train, cv=kfold, scoring='accuracy')
print(cv_results)
print(cv_results.mean())

[ 0.98901099  0.98901099  1.          0.98888889  0.98888889  1.          1.
  0.98888889  0.98888889  0.98888889]
0.992246642247


### Step 3: Read Sensor Data (Test Data) from Amazon Oracle DB that needs to be Classified using KNN

In [253]:
# Read equipment1 sensor data that need to be classified
# select statemet to query equipment1 sensor data
eqp1TestDataSQL="SELECT RECORD_ID,INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS FROM SENSOR_TEST_DATA WHERE UPPER(EQP_NAME)=UPPER('Vacuum Blower') "
cursor.execute(eqp1TestDataSQL)
eqp1SqlTestData = cursor.fetchall() 
eqp1TestDataList = [list(j) for j in eqp1SqlTestData]
eqp1TestDatatArray = np.asarray(eqp1TestDataList)


### Step 4: Predict the labels

In [254]:
eqp1UnknownLabels = eqp1TestDatatArray[:,1:8]
ids = eqp1TestDatatArray[:,0]
predictions = eqp1KNN.predict(eqp1UnknownLabels)

### Step 5: Update the results back to the Amazon DB

In [255]:
resultLabelsParam = cursor.arrayvar(cx_Oracle.STRING, predictions.tolist())
idsParam = cursor.arrayvar(cx_Oracle.NUMBER, ids.tolist())
args = [idsParam,resultLabelsParam]
# call pl/sql procedure which updates Sensor table data with predictions
cursor.callproc("BIG_DATA_503_PRJ_PKG.update_testdata_labels", args)

[[484666.0,
  484667.0,
  484668.0,
  484669.0,
  484670.0,
  484671.0,
  484672.0,
  484673.0,
  484674.0,
  484675.0,
  484676.0,
  484677.0,
  484678.0,
  484679.0,
  484680.0,
  484681.0,
  484682.0,
  484683.0,
  484684.0,
  484685.0,
  484686.0,
  484687.0,
  484688.0,
  484689.0,
  484690.0,
  484691.0,
  484692.0,
  484693.0,
  484694.0,
  484695.0,
  484696.0,
  484697.0,
  484698.0,
  484699.0,
  484700.0,
  484701.0,
  484702.0,
  484703.0,
  484704.0,
  484705.0,
  484706.0,
  484707.0,
  484708.0,
  484709.0,
  484710.0,
  484711.0,
  484712.0,
  484713.0,
  484714.0,
  484715.0,
  484716.0,
  484717.0,
  484718.0,
  484719.0,
  484720.0,
  484721.0,
  484722.0,
  484723.0,
  484724.0,
  484725.0,
  484726.0,
  484727.0,
  484729.0,
  484730.0,
  484731.0,
  484732.0,
  484734.0,
  484735.0,
  484736.0,
  484737.0,
  484739.0,
  484740.0,
  484741.0,
  484743.0,
  484744.0,
  484745.0,
  484746.0,
  484748.0,
  484749.0,
  484750.0,
  484751.0,
  484753.0,
  484754.0,
  48

### Step 6: Repeat above steps for Equipment2 

In [256]:
#Get Equipment2: Smart Valve Data
cursor.execute(eqp2SQL)
eqp2SQLResult = cursor.fetchall() 
eqp2List = [list(i) for i in eqp2SQLResult]
eqp2TrainArray = np.asarray(eqp2List)
# Split-out validation dataset for EQP2
x2 = eqp2TrainArray[:,0:7]
y2 = eqp2TrainArray[:,7]
x2_train, x2_validation, y2_train, y2_validation = model_selection.train_test_split(x2, y2, test_size=.2, random_state=7)
eqp2KNN = KNeighborsClassifier(n_neighbors=best_k)
eqp2KNN.fit(x2_train, y2_train)
# Read equipment2 sensor data that need to be classified
# select statemet to query equipment1 sensor data
eqp2TestDataSQL="SELECT RECORD_ID,INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS FROM SENSOR_TEST_DATA WHERE UPPER(EQP_NAME)=UPPER('Smart Valve') "
cursor.execute(eqp2TestDataSQL)
eqp2SqlTestData = cursor.fetchall() 
eqp2TestDataList = [list(j) for j in eqp2SqlTestData]
eqp2TestDatatArray = np.asarray(eqp2TestDataList)
eqp2UnknownLabels = eqp2TestDatatArray[:,1:8]
ids = eqp2TestDatatArray[:,0]
# predict the result labels
predictions = eqp2KNN.predict(eqp2UnknownLabels)
resultLabelsParam = cursor.arrayvar(cx_Oracle.STRING, predictions.tolist())
idsParam = cursor.arrayvar(cx_Oracle.NUMBER, ids.tolist())
args = [idsParam,resultLabelsParam]
# call pl/sql procedure which updates Sensor table data with predictions
cursor.callproc("BIG_DATA_503_PRJ_PKG.update_testdata_labels", args)

[[484155.0,
  484161.0,
  484167.0,
  484173.0,
  484178.0,
  484219.0,
  484224.0,
  484229.0,
  484235.0,
  484240.0,
  484246.0,
  484252.0,
  484299.0,
  484304.0,
  484308.0,
  484313.0,
  484318.0,
  484323.0,
  484372.0,
  484378.0,
  484384.0,
  484389.0,
  484395.0,
  484156.0,
  484157.0,
  484158.0,
  484159.0,
  484160.0,
  484162.0,
  484163.0,
  484164.0,
  484165.0,
  484166.0,
  484168.0,
  484169.0,
  484170.0,
  484171.0,
  484172.0,
  484174.0,
  484175.0,
  484176.0,
  484177.0,
  484179.0,
  484180.0,
  484181.0,
  484182.0,
  484183.0,
  484184.0,
  484185.0,
  484186.0,
  484187.0,
  484188.0,
  484189.0,
  484190.0,
  484191.0,
  484192.0,
  484193.0,
  484194.0,
  484195.0,
  484196.0,
  484197.0,
  484198.0,
  484199.0,
  484200.0,
  484201.0,
  484202.0,
  484203.0,
  484204.0,
  484205.0,
  484206.0,
  484207.0,
  484208.0,
  484209.0,
  484210.0,
  484211.0,
  484212.0,
  484213.0,
  484214.0,
  484215.0,
  484216.0,
  484217.0,
  484218.0,
  484220.0,
  48

### Step 7: Repeat above steps for Equipment3

In [257]:
#Get Equipment3:Air Compressor Data
cursor.execute(eqp3SQL)
eqp3SQLResult = cursor.fetchall() 
eqp3List = [list(i) for i in eqp3SQLResult]
eqp3TrainArray = np.asarray(eqp3List)
# Split-out validation dataset for EQP2
x3 = eqp3TrainArray[:,0:7]
y3 = eqp3TrainArray[:,7]
x3_train, x3_validation, y3_train, y3_validation = model_selection.train_test_split(x3, y3, test_size=.2, random_state=7)
eqp3KNN = KNeighborsClassifier(n_neighbors=best_k)
eqp3KNN.fit(x3_train, y3_train)
# Read equipment3 sensor data that need to be classified
# select statemet to query equipment1 sensor data
eqp3TestDataSQL="SELECT RECORD_ID,INT_AMB_TEMP,EXT_AMB_TEMP,INPUT_CUR,INPUT_VOLT,VIB_X_AXIS,VIB_Y_AXIS,VIB_Z_AXIS FROM SENSOR_TEST_DATA WHERE UPPER(EQP_NAME)=UPPER('Air Compressor') "
cursor.execute(eqp3TestDataSQL)
eqp3SqlTestData = cursor.fetchall() 
eqp3TestDataList = [list(j) for j in eqp3SqlTestData]
eqp3TestDatatArray = np.asarray(eqp3TestDataList)
eqp3UnknownLabels = eqp3TestDatatArray[:,1:8]
ids = eqp3TestDatatArray[:,0]
# predict the result labels
predictions = eqp3KNN.predict(eqp2UnknownLabels)
resultLabelsParam = cursor.arrayvar(cx_Oracle.STRING, predictions.tolist())
idsParam = cursor.arrayvar(cx_Oracle.NUMBER, ids.tolist())
args = [idsParam,resultLabelsParam]
# call pl/sql procedure which updates Sensor table data with predictions
cursor.callproc("BIG_DATA_503_PRJ_PKG.update_testdata_labels", args)

[[484543.0,
  484544.0,
  484545.0,
  484546.0,
  484547.0,
  484548.0,
  484549.0,
  484550.0,
  484551.0,
  484552.0,
  484553.0,
  484554.0,
  484555.0,
  484556.0,
  484557.0,
  484558.0,
  484559.0,
  484560.0,
  484561.0,
  484562.0,
  484563.0,
  484564.0,
  484565.0,
  484566.0,
  484567.0,
  484568.0,
  484569.0,
  484570.0,
  484571.0,
  484572.0,
  484573.0,
  484574.0,
  484575.0,
  484576.0,
  484577.0,
  484578.0,
  484579.0,
  484580.0,
  484581.0,
  484582.0,
  484583.0,
  484584.0,
  484585.0,
  484586.0,
  484587.0,
  484588.0,
  484589.0,
  484591.0,
  484592.0,
  484593.0,
  484594.0,
  484595.0,
  484597.0,
  484598.0,
  484599.0,
  484601.0,
  484602.0,
  484603.0,
  484604.0,
  484606.0,
  484607.0,
  484608.0,
  484609.0,
  484611.0,
  484612.0,
  484613.0,
  484614.0,
  484615.0,
  484616.0,
  484617.0,
  484618.0,
  484619.0,
  484620.0,
  484621.0,
  484622.0,
  484623.0,
  484624.0,
  484625.0,
  484626.0,
  484627.0,
  484628.0,
  484629.0,
  484630.0,
  48

In [258]:
# close oracle connections
cursor.close()
connection.close()
print('Done')

Done
